In [1]:
# 1. Define the model architecture and load the weights
# 2. Create the test data
# 3. Make predictions for the test videos
# 4. Finally, evaluate the model

In [2]:
from keras.models import Sequential, load_model
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

import numpy as np
import pandas as pd
import cv2
import math
import os
import h5py

from glob import glob
from scipy import stats as s

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

### Defining model architecture and weights 

In [16]:
# it will be similar to what we had while training the model
# base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) 

In [17]:
# fine-tuning the model architecture
model = Sequential()
model.add(InputLayer((24*3,)))    # input layer
model.add(Dense(units=10, activation='sigmoid', input_shape=(24*3,))) # hidden layer
model.add(Dense(2, activation='softmax'))    # output layer

In [11]:
# loading the trained weights
# model.load_weights("weights.hdf5")
model = load_model('weight.hdf5')

When compiling the model, make sure the loss function, optimizer and the metrics are the same as the ones used while training the model

In [18]:
# compiling the model
model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

### Generating the test data

In [19]:
# getting the test list
f = open("testlist_tennis.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating the dataframe
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.head()

,video_name
0,4_0.mp4
1,5_0.mp4


In [20]:
# creating the tags
train = pd.read_csv('myVids/train_new.csv')
y = train['class']
y = pd.get_dummies(y)

### Generating predictions for test videos
1. Create two empty lists – one to store the predictions and the other to store the actual tags
2. Take each video from the test set, extract frames for this video and store it in a folder (create a folder named temp in the current directory to store the frames). We will remove all the other files from this folder at each iteration
3. Read all the frames from the temp folder, extract features for these frames using the pre-trained model, predict tags, and then take the mode to assign a tag for that particular video and append it in the list
4. Append actual tags for each video in the second list

In [21]:
# creating two lists to store predicted and actual tags
predict = []
actual = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    cap = cv2.VideoCapture('myVids/'+videoFile)   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    # removing all other files from the temp folder
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames of this particular video in temp folder
            filename ='temp/' + "_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    
    # reading all the frames from temp folder
    images = glob("temp/*.jpg")
    
    prediction_images = []
    for i in tqdm(range(len(images))):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        prediction_images.append(img)
   
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    
#     image = []
#     for i in range(0,prediction_images.shape[0]): 
#         a = resize(prediction_images[i], preserve_range=True, output_shape=(224,224)).astype(int)      # reshaping to 224*224*3
#         image.append(a)
#     prediction_images = np.array(image)
    
#     prediction_images = preprocess_input(prediction_images, mode='tf')
    
#     # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    
#     # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)
#     # predicting tags for each array
    prediction = model.predict_classes(prediction_images)
#     # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
#     # appending the actual tag of the video
#     actual.append(videoFile.split('/')[0].split('_')[1])
    actual.append(videoFile.split(" ")[0].split("_")[1].split(".")[0])
    actual = list(map(int, actual))

  0%|          | 0/2 [00:04<?, ?it/s]


ValueError: Error when checking input: expected input_4 to have shape (72,) but got array with shape (25088,)

### Evaluating accuracy of the model

In [ ]:
# checking the accuracy of the predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100